In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import language_modelling_seq2seq

In [2]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(language_modelling_seq2seq)    
net = language_modelling_seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, 
                                            decoder_hidden_units = 100)
net.to(device)
net.fit(source_1, target_1, epochs = 5, batch_size = 150, lr = 0.0001, verbose = 3)

Net parameters: 207,195



Epoch:    1, Loss: 2.5494
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' ee  ne  e', ' e  ee  e ', '   ee  e e', '  ne  e e ', ' te  e e  ']
greedy_search
['          ', '          ', '          ', '          ', '          ']
sample
['itlaeulcte', 'acaher aos', '5s ti e  h', 'ankRs oarn', 'lhatR\ngaQ.']
beam_search
[['          ', '          ', '          ', '          ', '          '],
 ['e         ', 'e         ', 'e         ', 'e         ', '        . '],
 ['        . ', '         a', '         a', '        . ', '         .'],
 ['         a', '         t', '         t', '         .', 'e         '],
 ['         .', '         h', '         e', '         a', '         K']]
tensor([-11.0200, -10.8900, -10.5400,  -8.7100,  -8.1700], device='cuda:0')
tensor([[-11.0200, -10.8900, -10.5400,  -8.7100,  -8.1700],
        [-13.0200, -12.9400, -12.8000, -11.1400, -10.4800],
        [-13.4700, -13.5200, -13.3000, -11.5600, -10.5300],
        [-13.5600, -13.6


Epoch:    2, Loss: 1.9259
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e aone te', 'ehaane te ', 'etote te e', '  ne  e ea', ' n   e eet']
greedy_search
[' the the t', 'e the the ', 'e the the ', '  o       ', '  o       ']
sample
['iclungv th', 'rblaaytin,', 'or oack a ', 'ebd apsati', "ove'p\non t"]
beam_search
[['s the the ', 'e the the ', 'e the the ', 'e the the ', 'e the the '],
 [' the the t', 'on the the', 'on the the', 'the the th', ' the the t'],
 [' the the  ', 'the the th', 'in the the', 'e the ther', 'the the th'],
 [' the thes ', 'e the ther', 'ing the th', 'the the to', ' the there'],
 [' the the p', 'the the an', 'e the ther', 'the the  o', ' the thes ']]
tensor([-10.9800, -11.2600, -10.5500, -12.4100, -14.5600], device='cuda:0')
tensor([[-10.7700, -11.2600, -10.5500, -10.5700, -10.7700],
        [-10.9800, -11.5800, -11.2300, -11.1400, -11.1000],
        [-11.5300, -11.9000, -11.6700, -12.1300, -11.4600],
        [-11.7600, -12.8


Epoch:    3, Loss: 1.6384
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' h outerte', 'ehaute te ', 'eoate  e  ', 'hate  e   ', 'eler e   t']
greedy_search
[' the pare ', 'e the pare', 'e the pare', 'he the par', 'e the the ']
sample
['ly beces m', 'hisxWhimq ', 'piags\n\n\n  ', 'r ovonterb', ' doxd in t']
beam_search
[[' the there', 'e the the ', 'e the the ', 'he the the', 'he the the'],
 [' the the p', 'on the the', 'he the the', 'his the pa', 'hing the t'],
 [' the the c', 'e the ther', 'e the ther', 'he there t', 'hing the p'],
 [' the the t', 'the the pa', 'e the the\n', 'he the tha', 'he theres '],
 [' the the m', 'the the an', 'e the thee', 'he the and', 'hing the a']]
tensor([-12.9100, -12.9600, -11.6600, -11.9100, -11.2800], device='cuda:0')
tensor([[-10.7600, -10.6900,  -9.3500,  -9.8100, -11.0000],
        [-10.9800, -10.7200,  -9.9700, -11.4900, -11.9400],
        [-11.0100, -11.7900, -10.6800, -11.5100, -12.0800],
        [-11.0800, -1


Epoch:    4, Loss: 1.4955
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' h outerte', 'thouterte ', 'eouterte  ', 'outerte   ', 'orertene t']
greedy_search
[' the pawn ', 'the pawn t', 'e the pawn', 'of the paw', 'on the paw']
sample
['ve: If.xBe', "ne (hess's", 'fenteveins', 'touching t', 'hy Riage. ']
beam_search
[['ng the Bla', 'n the pawn', 'e the the ', 'the King t', 'and the pa'],
 ['ng the Kt-', 'e the pawn', ' of the th', 'the there ', 'his the pa'],
 ['s the Kt-K', 'the the pa', ' of the pa', 'the the pa', 'he there t'],
 ['s the Kt-B', 'n the pare', ' of the Kt', 'the King o', 'his there '],
 ['ng the pra', 'n the cant', 'e the ther', 'he there t', 'his theres']]
tensor([-11.3000, -13.2900, -10.5900, -11.0800, -12.8500], device='cuda:0')
tensor([[-10.0900, -10.7200,  -9.7800, -10.5600, -13.1000],
        [-11.2400, -11.4900, -10.2000, -10.8000, -13.1500],
        [-11.4300, -12.0600, -10.3300, -11.3200, -13.1900],
        [-11.5300, -12.46


Epoch:    5, Loss: 1.4107
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['nh outer e', 't outenten', ' outer en ', 'outer en  ', 'orertene t']
greedy_search
['ng the pre', 't of the p', ' of the pa', 'of the paw', 'ore the pa']
sample
['vion, of t', '1; Duthe, ', 'e ray thin', 'mates the ', 'ove\npreye,']
beam_search
[['ng the Bla', 't of the p', " the King'", 'the Black ', 'inger in t'],
 ['ng the Kin', 'e the pawn', ' the King ', "the King's", 'inger and '],
 ['ng the pre', 'k the pawn', ' of the pa', "the Black'", 'his there '],
 ['ng the Kt-', 't of the K', ' of the Bl', 'of the paw', 'inger inge'],
 ['ng the paw', 't of there', ' of the Kt', 'the Black\n', 'his theres']]
tensor([-10.8000, -10.0800,  -9.6300, -10.1700, -13.9800], device='cuda:0')
tensor([[ -9.6500, -10.0800,  -9.3700,  -7.7300, -12.1800],
        [-10.4000, -10.6000,  -9.6200,  -8.6000, -12.2400],
        [-10.8000, -10.9000,  -9.6300,  -8.6400, -12.8600],
        [-11.0500, -11.

In [9]:
[net.tensor2text(t) for t in net.beam_search(source_1[:10000], verbose = 1, batch_size = 50)[0]]

[["<START>ng the Black's Black",
  "<START>ng the Black's King ",
  "<START>ng the Black's there",
  "<START>ng the Black's King'",
  "<START>ng the Black's the p"],
 ['<START>t of the pawnt of th',
  '<START>t of the pawn, and t',
  '<START>t of the pawns the p',
  '<START>t of the pawnt on th',
  '<START>t of the pawn, and a'],
 ["<START> of the Black's the ",
  "<START> of the Black's King",
  "<START> of the Black's Blac",
  "<START> of the Black's ther",
  "<START> of the Black's the\n"],
 ["<START>the Black's King.\n\n ",
  "<START>the Black's King the",
  "<START>the Black's King's t",
  "<START>the Black's King's K",
  "<START>the Black's King's a"],
 ['<START>inger in the pawning',
  '<START>inger in the pawn.\n\n',
  '<START>inger in the pawn. T',
  '<START>inger in the pawn.  ',
  '<START>inger in the pawn. I'],
 ['<START>res the prenting the',
  '<START>res the playen the p',
  '<START>res the playen there',
  '<START>res the playen the K',
  '<START>res the playen the m'],